# auxiliar

In [2]:
import pandas as pd

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

In [4]:
tipificacion=pd.read_csv("/content/PRIORIZAR/data/tipificacion.csv").drop_duplicates()

tipificacion=tipificacion.reset_index()
tipificacion


,index,NUMERO_PQR,DESCRIPCION_PQR_x,correccion
0,0,MDM-PQR-35424371,SG//LT: 6016133556 cuenta 12054410753// client...,otros
1,1,MDM-PQR-35424385,SG//LT: 6017257398 cuenta 12054364769// client...,otros
2,2,MDM-PQR-35424468,SG//LT: 6013835268 cuenta 12053820540// client...,otros
3,3,MDM-PQR-35425107,Línea: 6016456888 Nombre de quien se comunica:...,SVAS
4,4,MDM-PQR-35426399,CLIENTE DESEA CONOCER LA FACTURA GENERADA,otros
...,...,...,...,...
1293,1391,MDM-PQR-37420844,CLIENTE CONSULTA POR AUMENTO EN SU FACTURA,INCREMENTO DE TARIFA
1294,1393,MDM-PQR-37421702,Nombre:JOSE TINJACA CANTOR Cedula:79452363 ...,SVAS
1295,1395,MDM-PQR-37423688,LINEA: 6017388873 NOMBRE: Nicolás Trujillo C...,otros
1296,1397,MDM-PQR-37424946,MIREYA VARGAS HERNANDEZ 63341371 CC 60179234...,otros


In [5]:
tipificacion['correccion'].value_counts()

otros                     467
INCREMENTO DE TARIFA      318
ASEGURAMIENTO             217
SVAS                      174
BENEFICIO DE RETENCION    122
Name: correccion, dtype: int64

In [6]:
import pandas as pd

import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification





nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# aca empieza el modelo

In [8]:
sys.path.append('../librerias')


In [9]:
from clean_text import clean_text

In [10]:

#declare answers and questions
descripcion=tipificacion.DESCRIPCION_PQR_x	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]

clean_descripcion  [881]

'solicito cancelacion del winsport pero le llego cobro completo el  pasado y se lo retiraron enseguida no entiende porque se le cobraron'

spelling

In [11]:
from spelling import correct_sentence


corrected_text_list = [correct_sentence(text) for text in clean_descripcion]
clean_descripcion=corrected_text_list

hola


In [ ]:
clean_descripcion[881]

'solicito cancelacion del winsport pero le llego cobro completo el pasado y se lo retiraron enseguida no entiende porque se le cobraron'

lemma token 

In [24]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[3]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['llego', 'cobro', 'hbo']

In [25]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


tipificacion=pd.concat([tipificacion, merged_df], axis=1).loc[:, ['descripcion', 'correccion']]

#merged_df = pd.merge(merged_df,leo , on='index', how='inner').loc[:, ['descripcion', 'correccion']]
#############################################

tipificacion = tipificacion[tipificacion['correccion'] != 'SVAS']


tipificacion

,descripcion,correccion
0,solicitar ajuste factura acuerdo cobro reconex...,otros
1,solicitar ajuste factura acuerdo cobro reconex...,otros
2,solicitar ajuste factura acuerdo cobro reconex...,otros
4,desear conocer factura generado,otros
6,aplicacion tarifa retencion ofrecer bajo ofert...,ASEGURAMIENTO
...,...,...
1292,factura incremento solicitar cancelacion ofrec...,INCREMENTO DE TARIFA
1293,consulta aumento factura,INCREMENTO DE TARIFA
1295,llego factura tenia llegar,otros
1296,mierdo tenia promocion aplicar,otros


In [26]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Assuming you have a DataFrame called 'data' with 'categories' column
# where categories are labeled as 1, 2, and 3
mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3}

# Separate features and target variable
X = tipificacion.drop('correccion', axis=1)
y = tipificacion['correccion'].replace(mapping)

# Oversampling using RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto')
X_resampled, y_resampled = oversampler.fit_resample(X, y)

mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3}

crear matriz de numeros 

In [28]:
from tfidf import TFIDF

In [29]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = X_resampled['descripcion']

y = y_resampled




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.95  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)



tf-idf with 1109 features
train with old features:  (1494, 1109)
train with new features: (1494, 361)
test with old features:  (374, 1109)
test with new features: (374, 361)
Training Data : (1494, 1109)
Testing Data :  (374, 1109)


In [30]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


In [31]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score



In [32]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "n_estimators": trial.suggest_int("n_estimators", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 350),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)


In [34]:
print('Best parameters:', study.best_params)

Best parameters: {'n_estimators': 361, 'lambda_l1': 0.39291873964268126, 'lambda_l2': 0.7561143520461611, 'num_leaves': 147, 'feature_fraction': 0.8675038227110754, 'bagging_fraction': 0.6037067082231619, 'bagging_freq': 7, 'learning_rate': 0.07529945129406222, 'min_child_samples': 61}


In [35]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

[LightGBM] [Warning] lambda_l1 is set=0.39291873964268126, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.39291873964268126
[LightGBM] [Warning] bagging_fraction is set=0.6037067082231619, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6037067082231619
[LightGBM] [Warning] lambda_l2 is set=0.7561143520461611, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7561143520461611
[LightGBM] [Warning] feature_fraction is set=0.8675038227110754, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8675038227110754
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


LGBMClassifier(bagging_fraction=0.6037067082231619, bagging_freq=7,
               feature_fraction=0.8675038227110754,
               lambda_l1=0.39291873964268126, lambda_l2=0.7561143520461611,
               learning_rate=0.07529945129406222, min_child_samples=61,
               n_estimators=361, num_leaves=147)

In [36]:
y_pred = mode.predict(X_test_new)


In [37]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.86      0.76      0.81        92
           1       0.82      0.86      0.84        94
           2       0.98      1.00      0.99        89
           3       0.95      0.99      0.97        99

    accuracy                           0.90       374
   macro avg       0.90      0.90      0.90       374
weighted avg       0.90      0.90      0.90       374



In [38]:
from joblib import dump


In [39]:
dump(value=mode, filename="content/retipificacion.joblib")

['content/retipificacion.joblib']